In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)


2024-02-21 02:49:51.015910: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 02:49:51.015954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 02:49:51.016473: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-21 02:49:51.019701: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 02:49:51.495955: W tensorflow/compiler/tf2

1 Physical GPUs, 1 Logical GPUs


2024-02-21 02:49:52.116941: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 02:49:52.141826: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 02:49:52.141878: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 02:49:52.145105: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-21 02:49:52.145149: I external/local_xla/xla/stream_executor

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

df = pd.read_csv('diabetes_prediction_dataset.csv')
df = df[df['age']>=18]
df = df[df['bmi']<=40]
#Preprocess the data
numeric_col=[]
non_numeric_col=[]
for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        if(df[column].nunique()<5):
            non_numeric_col.append(column)
        else:
            numeric_col.append(column)
    else:
        non_numeric_col.append(column)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
df['smoking_history'] = df['smoking_history'].replace({'not current':'former','ever':'never'})
df_copy = df.copy()
for col in non_numeric_col:
    df[col]=le.fit_transform(df[col])



In [3]:
y = df['diabetes']
X = df.drop('diabetes', axis = 1)

In [4]:
scale_pos_weight = y.value_counts()[0] /  y.value_counts()[1]

In [5]:
#Normalize the data
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, stratify = y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, stratify = y_test)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)
combined_X_train = np.concatenate((X_train_scaled,X_val_scaled), axis = 0)
combined_y_train = np.concatenate((y_train,y_val), axis = 0)

In [6]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_scaled, y_train = smote.fit_resample(X_train_scaled, y_train)

In [7]:
print(combined_X_train.shape)

(66539, 8)


In [8]:
from keras.models import Model
from keras.layers import Dense, BatchNormalization, Dropout, Input
from keras.saving import register_keras_serializable

@register_keras_serializable()
class DiabetesClassifier(Model):
    def __init__(self, data_input, num_of_dense_layers = 3, dense_number = 8, dropout = 0, l2 = 0.001, opt_threshold = 0):
        super(DiabetesClassifier, self).__init__()
        self.opt_threshold = opt_threshold
        self.num_of_dense_layers = num_of_dense_layers
        self.dense_number = dense_number
        self.dropout = dropout
        self.data_input = data_input
        self.l2 = l2
        self.model = self.build_model()
    
    def call(self, inputs):
        return self.model(inputs)
    
    def build_model(self):
        inp = Input(shape = self.data_input)
        reg = keras.regularizers.l2(self.l2)
        x = Dense(self.data_input, activation = 'relu', kernel_regularizer=reg)(inp)
        for i in range(self.num_of_dense_layers):
            x = Dense(units = self.dense_number, activation = 'relu', kernel_regularizer=reg)(x)
            x = BatchNormalization()(x)
            x = Dropout(self.dropout)(x)
        output = Dense(units =1, activation = 'sigmoid')(x)
        model = Model(inputs = inp, outputs = output)
        return model

In [9]:
from keras import metrics
import keras

In [39]:
model = DiabetesClassifier(X_train_scaled.shape[1], num_of_dense_layers =5, dense_number =16, dropout = 0.2, l2 = 0.001)
tensorboard = keras.callbacks.TensorBoard(log_dir='logs')
optimizer = keras.optimizers.Adam(0.001)
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = [metrics.AUC(), metrics.Precision(), metrics.Recall(), metrics.Accuracy()])
model.fit(X_train_scaled, y_train, validation_data = (X_val_scaled, y_val), batch_size =32, epochs = 20, class_weight = {1: scale_pos_weight, 0: 1}, callbacks = [tensorboard])



Epoch 1/20
3109/3109 [==============================] - 64s 20ms/step - loss: 1.3005 - auc_2: 0.8605 - precision_2: 0.6198 - recall_2: 0.9667 - accuracy: 1.0053e-05 - val_loss: 0.9896 - val_auc_2: 0.9514 - val_precision_2: 0.1836 - val_recall_2: 0.9954 - val_accuracy: 0.0000e+00
Epoch 2/20
3109/3109 [==============================] - 59s 19ms/step - loss: 0.8791 - auc_2: 0.9309 - precision_2: 0.6835 - recall_2: 0.9898 - accuracy: 2.0107e-05 - val_loss: 0.8389 - val_auc_2: 0.9546 - val_precision_2: 0.1934 - val_recall_2: 0.9991 - val_accuracy: 0.0000e+00
Epoch 3/20
3109/3109 [==============================] - 62s 20ms/step - loss: 0.7988 - auc_2: 0.9382 - precision_2: 0.7069 - recall_2: 0.9919 - accuracy: 0.0000e+00 - val_loss: 0.9485 - val_auc_2: 0.9582 - val_precision_2: 0.1931 - val_recall_2: 1.0000 - val_accuracy: 0.0000e+00
Epoch 4/20
3109/3109 [==============================] - 63s 20ms/step - loss: 0.7603 - auc_2: 0.9420 - precision_2: 0.7142 - recall_2: 0.9926 - accuracy: 0.0000

In [45]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve
n_splits = 5  # Number of folds for StratifiedKFold
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

opt_thresholds = []
f1_scores = []

for train_index, val_index in cv.split(combined_X_train, combined_y_train):
    # Split data into training and validation for the current fold
    X_train_curr, X_val_curr = combined_X_train[train_index], combined_X_train[val_index]
    y_train_curr, y_val_curr = combined_y_train[train_index], combined_y_train[val_index]

    # Predict probabilities for the positive class on the validation data
    y_pred_prob = model.predict(X_val_curr).ravel()

    # Compute precision-recall curve
    precision, recall, thresholds = precision_recall_curve(y_val_curr, y_pred_prob)

    # Calculate F1 scores for each threshold
    f1_scores_fold = [2 * (p * r) / (p + r) if (p + r) > 0 else 0 for p, r in zip(precision, recall)]

    # Find the index of the maximum F1 score
    opt_idx = np.argmax(f1_scores_fold)
    opt_threshold = thresholds[opt_idx] if opt_idx < len(thresholds) else 1.0

    # Store the optimal threshold for this fold
    opt_thresholds.append(opt_threshold)
    f1_scores.append(f1_scores_fold[opt_idx])

# Optionally, you can print or analyze the optimal thresholds and F1 scores for each fold
for i, (threshold, f1_score) in enumerate(zip(opt_thresholds, f1_scores)):
    print(f"Fold {i+1}: Optimal threshold: {threshold}, F1 Score: {f1_score}")

# You might also calculate and print the average optimal threshold and F1 score across folds if desired
average_opt_threshold = np.mean(opt_thresholds)
average_f1_score = np.mean(f1_scores)
print(f"Average Optimal Threshold: {average_opt_threshold}, Average F1 Score: {average_f1_score}")

416/416 [==============================] - 1s 3ms/step
Fold 1: Optimal threshold: 0.9860290884971619, F1 Score: 0.7775131014768937
Fold 2: Optimal threshold: 0.9865700602531433, F1 Score: 0.8
Fold 3: Optimal threshold: 0.9863243699073792, F1 Score: 0.7822349570200574
Fold 4: Optimal threshold: 0.9865864515304565, F1 Score: 0.7896226415094341
Fold 5: Optimal threshold: 0.9822980165481567, F1 Score: 0.7978142076502731
Average Optimal Threshold: 0.9855615496635437, Average F1 Score: 0.7894369815313317


In [46]:
y_pred_prob = (model.predict(X_val_scaled).ravel() >= average_opt_threshold).astype(int)
print(classification_report(y_val, y_pred_prob))
y_pred_prob = (model.predict(X_test_scaled).ravel() >= average_opt_threshold).astype(int)
print(classification_report(y_test, y_pred_prob))

367/367 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     10658
           1       0.93      0.67      0.78      1085

    accuracy                           0.97     11743
   macro avg       0.95      0.83      0.88     11743
weighted avg       0.96      0.97      0.96     11743

367/367 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

           0       0.97      1.00      0.98     10658
           1       0.93      0.67      0.78      1084

    accuracy                           0.96     11742
   macro avg       0.95      0.83      0.88     11742
weighted avg       0.96      0.96      0.96     11742


In [14]:
import optuna
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_recall_curve, f1_score


def objective(trial):
    num_of_layers = trial.suggest_int('num_of_layers', 2, 6, step = 1)
    dense_num_of_neurons = trial.suggest_int('num_of_neurons', 1, 6, step = 1)
    dense_num_of_neurons = 2 ** dense_num_of_neurons
    dropout = trial.suggest_float('dropout', 0.2, 0.6)
    l2 = trial.suggest_float('l2', 0.001, 1, log = True)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 1, log = True)
    scale_pos_weight_multiplier = trial.suggest_int('scale_pos_weight_multiplier', 1, 4)
    
    model = DiabetesClassifier(X_train_scaled.shape[1], num_of_dense_layers =num_of_layers, dense_number =dense_num_of_neurons, dropout = dropout, l2 = l2)
    tensorboard = keras.callbacks.TensorBoard(log_dir='logs')
    optimizer = keras.optimizers.Adam(learning_rate = learning_rate)
    model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = [metrics.AUC(), metrics.Precision(), metrics.Recall(), metrics.Accuracy()])
    model.fit(X_train_scaled, y_train, validation_data = (X_val_scaled, y_val), batch_size =32, epochs = 20, class_weight = {1: scale_pos_weight/scale_pos_weight_multiplier, 0: 1}, callbacks = [tensorboard])

    n_splits = 5  # Number of folds for StratifiedKFold
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    opt_thresholds = []
    f1_scores = []
    
    for train_index, val_index in cv.split(combined_X_train, combined_y_train):
        # Split data into training and validation for the current fold
        X_train_curr, X_val_curr = combined_X_train[train_index], combined_X_train[val_index]
        y_train_curr, y_val_curr = combined_y_train[train_index], combined_y_train[val_index]
    
        # Predict probabilities for the positive class on the validation data
        y_pred_prob = model.predict(X_val_curr).ravel()
    
        # Compute precision-recall curve
        precision, recall, thresholds = precision_recall_curve(y_val_curr, y_pred_prob)
    
        # Calculate F1 scores for each threshold
        f1_scores_fold = [2 * (p * r) / (p + r) if (p + r) > 0 else 0 for p, r in zip(precision, recall)]
    
        # Find the index of the maximum F1 score
        opt_idx = np.argmax(f1_scores_fold)
        opt_threshold = thresholds[opt_idx] if opt_idx < len(thresholds) else 1.0
    
        # Store the optimal threshold for this fold
        opt_thresholds.append(opt_threshold)
        f1_scores.append(f1_scores_fold[opt_idx])
    
        average_opt_threshold = np.mean(opt_thresholds)
        average_f1_score = np.mean(f1_scores)
        trial.set_user_attr('average_opt_threshold', average_opt_threshold)
        return average_f1_score
    
study = optuna.create_study(direction='maximize')
study.sampler = optuna.samplers.TPESampler(multivariate=True)
study.optimize(objective, n_trials=20)

print('Number of finished trials:', len(study.trials))
print('Best result: ', study.best_trial.value)
print('Best trial:', study.best_trial.params)
# To use the best parameters:
best_params = study.best_trial.params
best_threshold = study.best_trial.user_attrs['average_opt_threshold']



[I 2024-02-21 03:36:00,842] A new study created in memory with name: no-name-9e4fc301-2b61-44e3-bba5-5fc21a3f4899


Epoch 1/20


/home/mamdouh/anaconda3/envs/GPU/lib/python3.10/site-packages/optuna/samplers/_tpe/sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(


3109/3109 [==============================] - 58s 18ms/step - loss: 2.7958 - auc_2: 0.8054 - precision_2: 0.5912 - recall_2: 0.9786 - accuracy: 0.0032 - val_loss: 3.1994 - val_auc_2: 0.9309 - val_precision_2: 0.1416 - val_recall_2: 0.9954 - val_accuracy: 0.0000e+00
Epoch 2/20
3109/3109 [==============================] - 59s 19ms/step - loss: 2.6481 - auc_2: 0.8187 - precision_2: 0.6009 - recall_2: 0.9798 - accuracy: 0.0013 - val_loss: 5.2230 - val_auc_2: 0.9417 - val_precision_2: 0.0924 - val_recall_2: 1.0000 - val_accuracy: 0.0000e+00
Epoch 3/20
3109/3109 [==============================] - 59s 19ms/step - loss: 2.5939 - auc_2: 0.8189 - precision_2: 0.5945 - recall_2: 0.9789 - accuracy: 5.1273e-04 - val_loss: 0.7920 - val_auc_2: 0.9286 - val_precision_2: 0.5632 - val_recall_2: 0.6608 - val_accuracy: 0.0000e+00
Epoch 4/20
3109/3109 [==============================] - 57s 18ms/step - loss: 2.2030 - auc_2: 0.8192 - precision_2: 0.5981 - recall_2: 0.9803 - accuracy: 7.5401e-04 - val_loss: 28

[I 2024-02-21 03:54:53,012] Trial 0 finished with value: 0.6777442094662639 and parameters: {'num_of_layers': 4, 'num_of_neurons': 4, 'dropout': 0.2634770756183719, 'l2': 0.0013749676843788107, 'learning_rate': 0.3525139685978232, 'scale_pos_weight_multiplier': 1}. Best is trial 0 with value: 0.6777442094662639.


...

Epoch 1/20
3109/3109 [==============================] - 43s 14ms/step - loss: 0.9567 - auc_20: 0.9180 - precision_20: 0.6489 - recall_20: 0.9856 - accuracy: 0.0010 - val_loss: 0.6651 - val_auc_20: 0.9472 - val_precision_20: 0.2265 - val_recall_20: 0.9696 - val_accuracy: 0.0000e+00
Epoch 2/20
3109/3109 [==============================] - 42s 13ms/step - loss: 0.9491 - auc_20: 0.9173 - precision_20: 0.6376 - recall_20: 0.9893 - accuracy: 0.0014 - val_loss: 0.6353 - val_auc_20: 0.9479 - val_precision_20: 0.2290 - val_recall_20: 0.9687 - val_accuracy: 0.0000e+00
Epoch 3/20
3109/3109 [==============================] - 42s 13ms/step - loss: 0.9453 - auc_20: 0.9183 - precision_20: 0.6382 - recall_20: 0.9896 - accuracy: 0.0016 - val_loss: 0.8226 - val_auc_20: 0.9461 - val_precision_20: 0.1818 - val_recall_20: 0.9899 - val_accuracy: 0.0000e+00
Epoch 4/20
3109/3109 [==============================] - 43s 14ms/step - loss: 0.9443 - auc_20: 0.9181 - precision_20: 0.6369 - recall_20: 0.9893 - accurac

[I 2024-02-21 09:23:43,102] Trial 18 finished with value: 0.7081252837040398 and parameters: {'num_of_layers': 2, 'num_of_neurons': 2, 'dropout': 0.30888000331299115, 'l2': 0.0020044763823247277, 'learning_rate': 0.01094851045002535, 'scale_pos_weight_multiplier': 1}. Best is trial 15 with value: 0.7578454332552694.


Epoch 1/20
3109/3109 [==============================] - 60s 19ms/step - loss: 1.4070 - auc_21: 0.8038 - precision_21: 0.5444 - recall_21: 0.9918 - accuracy: 3.2171e-04 - val_loss: 2.2019 - val_auc_21: 0.9369 - val_precision_21: 0.1005 - val_recall_21: 1.0000 - val_accuracy: 0.0000e+00
Epoch 2/20
3109/3109 [==============================] - 59s 19ms/step - loss: 1.3419 - auc_21: 0.8234 - precision_21: 0.5526 - recall_21: 0.9930 - accuracy: 4.2225e-04 - val_loss: 1.1155 - val_auc_21: 0.9370 - val_precision_21: 0.1471 - val_recall_21: 0.9963 - val_accuracy: 0.0000e+00
Epoch 3/20
3109/3109 [==============================] - 58s 19ms/step - loss: 1.3394 - auc_21: 0.8258 - precision_21: 0.5540 - recall_21: 0.9927 - accuracy: 8.0428e-04 - val_loss: 1.2354 - val_auc_21: 0.9418 - val_precision_21: 0.1260 - val_recall_21: 1.0000 - val_accuracy: 0.0000e+00
Epoch 4/20
3109/3109 [==============================] - 58s 19ms/step - loss: 1.3389 - auc_21: 0.8257 - precision_21: 0.5527 - recall_21: 0.99

[I 2024-02-21 09:43:11,055] Trial 19 finished with value: 0.7180180180180181 and parameters: {'num_of_layers': 4, 'num_of_neurons': 2, 'dropout': 0.42996801457879885, 'l2': 0.01320030349127959, 'learning_rate': 0.030581180961981713, 'scale_pos_weight_multiplier': 1}. Best is trial 15 with value: 0.7578454332552694.


Number of finished trials: 20
Best trial: {'num_of_layers': 2, 'num_of_neurons': 4, 'dropout': 0.3078955107209812, 'l2': 0.0034225062459207157, 'learning_rate': 0.001597106032793918, 'scale_pos_weight_multiplier': 1}


In [16]:
model = DiabetesClassifier(X_train_scaled.shape[1], num_of_dense_layers =best_params['num_of_layers'], dense_number = 2 ** best_params['num_of_neurons'], dropout = best_params['dropout'], l2 = best_params['l2'])
tensorboard = keras.callbacks.TensorBoard(log_dir='logs')
optimizer = keras.optimizers.Adam(learning_rate = best_params['learning_rate'])
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = [metrics.AUC(), metrics.Precision(), metrics.Recall(), metrics.Accuracy()])
model.fit(X_train_scaled, y_train, validation_data = (X_val_scaled, y_val), batch_size =32, epochs = 20, class_weight = {1: scale_pos_weight/best_params['scale_pos_weight_multiplier'], 0: 1}, callbacks = [tensorboard])

Epoch 1/20
3109/3109 [==============================] - 44s 14ms/step - loss: 1.0195 - auc_22: 0.9162 - precision_22: 0.6719 - recall_22: 0.9767 - accuracy: 3.0160e-05 - val_loss: 0.9082 - val_auc_22: 0.9466 - val_precision_22: 0.1861 - val_recall_22: 0.9908 - val_accuracy: 0.0000e+00
Epoch 2/20
3109/3109 [==============================] - 42s 14ms/step - loss: 0.8168 - auc_22: 0.9389 - precision_22: 0.6899 - recall_22: 0.9890 - accuracy: 2.2118e-04 - val_loss: 0.6336 - val_auc_22: 0.9459 - val_precision_22: 0.2354 - val_recall_22: 0.9604 - val_accuracy: 0.0000e+00
Epoch 3/20
3109/3109 [==============================] - 42s 14ms/step - loss: 0.7756 - auc_22: 0.9420 - precision_22: 0.7000 - recall_22: 0.9906 - accuracy: 2.7144e-04 - val_loss: 0.8337 - val_auc_22: 0.9503 - val_precision_22: 0.2144 - val_recall_22: 0.9843 - val_accuracy: 0.0000e+00
Epoch 4/20
3109/3109 [==============================] - 43s 14ms/step - loss: 0.7463 - auc_22: 0.9439 - precision_22: 0.7122 - recall_22: 0.99

In [17]:
y_pred_prob = (model.predict(X_val_scaled).ravel() >= best_threshold).astype(int)
print(classification_report(y_val, y_pred_prob))
y_pred_prob = (model.predict(X_test_scaled).ravel() >= best_threshold).astype(int)
print(classification_report(y_test, y_pred_prob))

367/367 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.96      0.99      0.97     10658
           1       0.83      0.58      0.69      1085

    accuracy                           0.95     11743
   macro avg       0.90      0.78      0.83     11743
weighted avg       0.95      0.95      0.95     11743

367/367 [==============================] - 1s 2ms/step
              precision    recall  f1-score   support

           0       0.96      0.99      0.98     10658
           1       0.86      0.61      0.72      1084

    accuracy                           0.96     11742
   macro avg       0.91      0.80      0.85     11742
weighted avg       0.95      0.96      0.95     11742


In [21]:
model.save('model')

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [ ]:
# Create a converter object
ae = tf.keras.models.load_model('model')
converter = tf.lite.TFLiteConverter.from_keras_model(ae)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]
# This example enables dynamic range quantization
converter.allow_custom_ops = True  # Allow for the possibility of custom operations

# Enable verbose logging
converter.experimental_new_converter = True
converter.optimizations = [tf.lite.Optimize.DEFAULT]


# Perform the conversion
tflite_model = converter.convert()
# Replace 'converted_model.tflite' with the desired path for your .tflite model
with open('converted_model.tflite', 'wb') as f:
    f.write(tflite_model)
